<a href="https://colab.research.google.com/github/ankitgoelcmu/DeepLearning/blob/main/RNN_Model01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, num_samples=1000, seq_len=10):
        self.data = []
        self.labels = []
        for i in range(num_samples):
            seq = np.random.randint(0, 2, seq_len)  # Binary sequence
            #print(f"sqe number : {i} : data : {seq}")
            label = int(np.sum(seq) % 2)  # 0: even sum, 1: odd sum
            self.data.append(seq)
            self.labels.append(label)
        #print (f"Self data: {self.data}")
        self.data = np.array(self.data, dtype=np.float32)
        self.labels = np.array(self.labels, dtype=np.long)
        print(f"data shape: {self.data.shape}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.labels[idx])

# Split train/test
full_dataset = SequenceDataset(num_samples=4000)
print(f"Full Dataset shape : {full_dataset.data.shape}")
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'Train samples: {len(train_dataset)}, Test: {len(test_dataset)}')
print(f'Sample sequence: {train_dataset[0][0][:5]}... Label: {train_dataset[0][1]}')



data shape: (4000, 10)
Full Dataset shape : (4000, 10)
Train samples: 3200, Test: 800
Sample sequence: tensor([1., 1., 1., 0., 1.])... Label: 1


In [ ]:
batch_idx, (data, target) = next(enumerate(train_loader))
print(f"Batch idx: {batch_idx}")
print(f"Data shape: {data.shape}")
print(f"Target shape: {target.shape}")
ex_tensor = data[batch_idx]
ex_tensor

Batch idx: 0
Data shape: torch.Size([32, 10])
Target shape: torch.Size([32])


tensor([0., 0., 1., 0., 1., 0., 0., 1., 1., 1.])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

class RNN_model_01(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, output_size):
        super().__init__()
        self.rnn_layer = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_layer = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()  # Optional: Apply after RNN if needed

    def forward(self, x):
        # Unsqueeze for input_size=1: [batch, seq] → [batch, seq, 1]
        if x.dim() == 2:
            x = x.unsqueeze(-1)

        # Single RNN call: Unpack tuple!
        rnn_out, _ = self.rnn_layer(x)  # rnn_out: [batch, seq, hidden]

        # Optional ReLU on output
        rnn_out = self.relu(rnn_out)

        # Last time step for classification: [batch, hidden]
        last_hidden = rnn_out[:, -1, :]

        # Linear: [batch, output_size]
        return self.fc_layer(last_hidden)

In [ ]:
model_01 = RNN_model_01 (1, 10,2,0.2,2)
model_01.to(device)

RNN_model_01(
  (rnn_layer): RNN(1, 10, num_layers=2, batch_first=True, dropout=0.2)
  (fc_layer): Linear(in_features=10, out_features=2, bias=True)
  (relu): ReLU()
)

In [ ]:

# define the loss function and optimizer

loss_fin = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_01.parameters(), lr=0.001)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_01.parameters(), lr=0.01)

def train(model, train_loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        print(f"Input shape: {data.shape}")  # e.g., [32, 10]

        optimizer.zero_grad()
        output = model(data)  # Fixed: [32, 2]
        print(f"Output shape: {output.shape}")

        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if batch_idx == 0:  # Print first batch only
            break  # Test one batch
    print(f"Avg Loss: {total_loss / len(train_loader):.4f}")

# Run one epoch
train(model_01, train_loader, loss_fn, optimizer, device)


Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0070


In [ ]:
import torch
from sklearn.metrics import accuracy_score  # If not imported

def evaluate_model(model, test_loader, loss_fn, device):
    """
    Evaluates the model on test data: Computes avg loss and accuracy.

    Args:
        model: Your RNN_model_01 instance.
        test_loader: Test DataLoader.
        loss_fn: Loss function (e.g., nn.CrossEntropyLoss()).
        device: 'cuda' or 'cpu'.

    Returns:
        dict: {'test_loss': float, 'test_accuracy': float}
    """
    model.eval()  # Eval mode (disables dropout)
    total_loss = 0.0
    all_preds = []  # For accuracy
    all_targets = []  # For accuracy

    with torch.inference_mode():  # No grads, efficient
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)  # [batch, output_size]

            # Loss
            loss = loss_fn(output, target)
            total_loss += loss.item() * len(data)  # Weighted by batch size (handles varying sizes)

            # Predictions for accuracy
            y_pred = torch.argmax(output, dim=1)  # [batch]
            all_preds.extend(y_pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    # Averages
    num_samples = len(test_loader.dataset)
    avg_loss = total_loss / num_samples  # Per-sample average
    accuracy = accuracy_score(all_targets, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")

    return {'test_loss': avg_loss, 'test_accuracy': accuracy}
#

In [ ]:
# After defining model, loss_fn, optimizer, train_loader, test_loader, device
epochs = 10
for epoch in range(epochs):
    # Train one epoch (your train function)
    train_loss = train(model_01, train_loader, loss_fn, optimizer, device)
    #print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}")

    # Eval every epoch (or if epoch % 2 == 0 for less freq)
    results = evaluate_model(model_01, test_loader, loss_fn, device)

    # Optional: Early stop if test_loss > threshold
    if results['test_accuracy'] > 0.95:
        print("High accuracy—stopping early!")
        break

Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0069
Test Loss: 0.6931, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0069
Test Loss: 0.6943, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0071
Test Loss: 0.6946, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0068
Test Loss: 0.6959, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0071
Test Loss: 0.6957, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0072
Test Loss: 0.6944, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0069
Test Loss: 0.6937, Test Accuracy: 0.5162
Input shape: torch.Size([32, 10])
Output shape: torch.Size([32, 2])
Avg Loss: 0.0070
Test Loss: 0.6931, Test Accuracy:

In [ ]:
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    output = model_01(data)
    y_pred_class = torch.argmax(output,dim=1)
    print(f"predictied  y_pred_class: {y_pred_class}")
    print(f"Test Label: {target}")


predictied  y_pred_class: tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 1])
Test Label: tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 0])
predictied  y_pred_class: tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1])
Test Label: tensor([1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0])
predictied  y_pred_class: tensor([1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 1])
Test Label: tensor([1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 1])
predictied  y_pred_class: tensor([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 1])
Test Label: tensor([0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1

In [ ]:
batch_idx, (data, target) = next(enumerate(train_loader))
print(f"Batch idx: {batch_idx}")
print(f"Data shape: {data.shape}")
print(f"data : {data}")
print(f" Data shape after unsqueeze: {data.unsqueeze(dim=2).shape}")
print(f"data aftwe unsqueeze : {data}")
print(f"Target shape: {target.shape}")
ex_tensor = data[batch_idx]
print(f"example tensor to understand input to RNN : {ex_tensor}")





Batch idx: 0
Data shape: torch.Size([32, 10])
data : tensor([[0., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0., 1., 0.],
        [0., 1., 1., 0., 1., 1., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1., 0., 1., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
        [0., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [1., 1., 0., 0., 0., 1., 1., 0., 1., 0.],
        [1., 0., 0., 1., 1., 1., 0., 1., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 0., 1., 1.

In [ ]:
# Dummy color sequence: [batch=1, seq_len=5 frames, input_size=3 channels per frame (flattened or per-pixel)]
x = torch.rand(32, 10, 1)  # Simplified; real would be [1, 5, 3, H, W] then flattened
rnn = nn.RNN(input_size=1, hidden_size=32, batch_first=True)
out, _ = rnn(x)
print(out.shape)  # [1, 5, 32] — processes 3 features per time ste

torch.Size([32, 10, 32])
